In [ ]:
; Definition for Mac, Read Only
; DEF.MAC.R/O
; DEFMACRO

# 宏的展开期和运行期

In [3]:
; 区别: 生成代码的代码(宏), 最终构成程序的代码
; 编写宏时, 是在编写那些将被编译器用来生成代码并随后编译的程序.
; 只有当所有的宏都被完全展开, 并且产生的代码被编译后, 程序才可以实际运行.
;
; 宏运行的时期: 宏展开期(macro expansion time)
; 运行期(runtime): 正常的代码实际运行的阶段
; 在宏展开期无法访问那些仅存于运行期的数据
;
; 总是向宏传递那些代表宏形式中的子形式的未经求值的Lisp对象
; 宏形式: 首元素为宏名的Lisp形式

# DEFMACRO

In [4]:
; (defmacro name (parameter*)
;   "Optional documentation string."
;   body-form*)

In [5]:
; 编写宏的步骤:
; (1) 编写示例的宏调用, 以及它应当展开生成的代码
; (2) 编写从示例调用的参数中生成手写展开式的代码
; (3) 确保宏抽象不产生蟹柳

In [6]:
; do-primes

(defun primep (number)
    (when (> number 1)
        (loop for fac from 2 to (isqrt number) never (zerop (mod number fac)))))

(defun next-prime (number)
    (loop for n from number when (primep n) return n))

; (do-primes (p 0 19)
;   (format t "~d " p))

(do ((p (next-prime 0) (next-prime (1+ p))))
    ((> p 19))
    (format t "~d " p))

PRIMEP

NEXT-PRIME

NIL

2 3 5 7 11 13 17 19 

# 宏形参

In [7]:
(defmacro do-primes (var-and-range &rest body) ; &body与&rest等价
    (let ((var (first var-and-range))
          (start (second var-and-range))
          (end (third var-and-range)))
         `(do ((,var (next-prime ,start) (next-prime (1+ ,var))))
              ((> ,var ,end))
              ,@body)))

DO-PRIMES

In [8]:
(do-primes (p 0 19)
    (format t "~d " p))

NIL

2 3 5 7 11 13 17 19 

In [12]:
(macroexpand-1 '(do-primes (p 0 19) (format t "~d " p)))

(DO ((P (NEXT-PRIME 0) (NEXT-PRIME (1+ P)))) ((> P 19)) (FORMAT T "~d " P))

T

In [16]:
; spike: 为什么需要求值
(defmacro do-primes2 (var-and-range &rest body)
    (let ((var (first var-and-range))
          (start (second var-and-range))
          (end (third var-and-range)))
         `(do ((var (next-prime start) (next-prime (1+ var))))
              ((> var end))
              ,@body)))
(macroexpand-1 '(do-primes2 (p 0 19) (format t "~d " p)))

DO-PRIMES2

(DO ((VAR (NEXT-PRIME START) (NEXT-PRIME (1+ VAR))))
    ((> VAR END))
  (FORMAT T "~d " P))

T


SIMPLE-STYLE-WARNING: 
  The variable VAR is defined but never used.

SIMPLE-STYLE-WARNING: 
  The variable START is defined but never used.

SIMPLE-STYLE-WARNING: 
  The variable END is defined but never used.

REDEFINITION-WITH-DEFMACRO: 
  redefining COMMON-LISP-USER::DO-PRIMES2 in DEFMACRO


In [18]:
; 宏形参列表是解构(destructuring)形参列表
(defmacro do-primes3 ((var start end) &rest body)
    `(do ((,var (next-prime ,start) (next-prime (1+ ,var))))
         ((> ,var ,end))
         ,@body))
(macroexpand-1 '(do-primes3 (p 0 19) (format t "~d " p)))

DO-PRIMES3

(DO ((P (NEXT-PRIME 0) (NEXT-PRIME (1+ P)))) ((> P 19)) (FORMAT T "~d " P))

T


REDEFINITION-WITH-DEFMACRO: 
  redefining COMMON-LISP-USER::DO-PRIMES3 in DEFMACRO


# 生成展开式

## 反引用(backquote)

In [19]:
; 反引用: 当读取器读到一个反引用表达式时, 将其翻译成生成适当列表结构的代码

In [27]:
`(a (+ 1 2) c)
(list 'a '(+ 1 2) 'c)

`(a ,(+ 1 2) c)
(list 'a (+ 1 2) 'c)

(A (+ 1 2) C)

(A (+ 1 2) C)

(A 3 C)

(A 3 C)

In [28]:
`(a (list 1 2) c)
(list 'a '(list 1 2) 'c)

`(a ,(list 1 2) c)
(list 'a (list 1 2) 'c)

`(a ,@(list 1 2) c)
(append (list 'a) (list 1 2) (list 'c))

(A (LIST 1 2) C)

(A (LIST 1 2) C)

(A (1 2) C)

(A (1 2) C)

(A 1 2 C)

(A 1 2 C)

## 解引用(unquote)

In [ ]:
; ,
; ,@

# 堵住漏洞

In [29]:
; 宏可能以三种方式泄露其内部工作细节

In [30]:
; (1) end多次求值
(macroexpand-1 '(do-primes (p 0 (random 100)) (format t "~d " p)))

(DO ((P (NEXT-PRIME 0) (NEXT-PRIME (1+ P))))
    ((> P (RANDOM 100)))
  (FORMAT T "~d " P))

T

In [40]:
; 修复(1) end只求值一次
; (2) end在var, start之前求值
; (3) ending-value的命名: 可能跟传递给宏的代码或宏被调用的上下文产生交互
(defmacro do-primes4 ((var start end) &rest body)
    `(do ((ending-value ,end)
          (,var (next-prime ,start) (next-prime (1+ ,var))))
         ((> ,var ending-value))
         ,@body))
(macroexpand-1 '(do-primes4 (p 0 (random 100)) (format t "~d " p)))
(do-primes4 (p 0 (random 100)) (format t "~d " p))

DO-PRIMES4

(DO ((ENDING-VALUE (RANDOM 100))
     (P (NEXT-PRIME 0) (NEXT-PRIME (1+ P))))
    ((> P ENDING-VALUE))
  (FORMAT T "~d " P))

T

NIL


REDEFINITION-WITH-DEFMACRO: 
  redefining COMMON-LISP-USER::DO-PRIMES4 in DEFMACRO
2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 

In [41]:
; 修复(2) 调整DO中变量初始化形式的顺序
(defmacro do-primes5 ((var start end) &rest body)
    `(do ((,var (next-prime ,start) (next-prime (1+ ,var)))
          (ending-value ,end))
         ((> ,var ending-value))
         ,@body))
(macroexpand-1 '(do-primes5 (p 0 (random 100)) (format t "~d " p)))
(do-primes5 (p 0 (random 100)) (format t "~d " p))

DO-PRIMES5

(DO ((P (NEXT-PRIME 0) (NEXT-PRIME (1+ P)))
     (ENDING-VALUE (RANDOM 100)))
    ((> P ENDING-VALUE))
  (FORMAT T "~d " P))

T

NIL


REDEFINITION-WITH-DEFMACRO: 
  redefining COMMON-LISP-USER::DO-PRIMES5 in DEFMACRO
2 3 5 

In [42]:
; 修复(3) 使用函数GENSYM
(defmacro do-primes6 ((var start end) &body body)
    (let ((ending-value-name (gensym)))
         `(do ((,var (next-prime ,start) (next-prime (1+ ,var)))
               (,ending-value-name ,end))
              ((> ,var ,ending-value-name))
              ,@body)))
(macroexpand-1 '(do-primes6 (p 0 (random 100)) (format t "~d " p)))
(do-primes6 (p 0 (random 100)) (format t "~d " p))

DO-PRIMES6

(DO ((P (NEXT-PRIME 0) (NEXT-PRIME (1+ P)))
     (#:G560 (RANDOM 100)))
    ((> P #:G560))
  (FORMAT T "~d " P))

T

NIL

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 

In [43]:
; 需要遵循的规则: 除非有特殊理由
; (1) 需要将展开式的任何子形式放在一个位置上, 使其求值顺序与宏调用的子形式相同
; (2) 需要确保子形式仅被求值一次
; (3) 在宏展开期使用GENSYM函数创建展开式中用到的变量名

# 用于编写宏的宏

In [44]:
; 一个模式: 以一个LET形式开始, 后者引入一些变量来保存宏展开过程中用到的生成符号.

In [46]:
(defmacro with-gensyms ((&rest names) &body body)
    `(let ,(loop for n in names collect `(,n (gensym)))
          ,@body))

WITH-GENSYMS


REDEFINITION-WITH-DEFMACRO: 
  redefining COMMON-LISP-USER::WITH-GENSYMS in DEFMACRO


In [47]:
(loop for n in '(a b c) collect `(,n (gensym)))

((A (GENSYM)) (B (GENSYM)) (C (GENSYM)))

In [48]:
(defmacro do-primes7 ((var start end) &body body)
    (with-gensyms (ending-value-name)
        `(do ((,var (next-prime ,start) (next-prime (1+ ,var)))
               (,ending-value-name ,end))
              ((> ,var ,ending-value-name))
              ,@body)))
(macroexpand-1 '(do-primes7 (p 0 (random 100)) (format t "~d " p)))
(do-primes7 (p 0 (random 100)) (format t "~d " p))

DO-PRIMES7

(DO ((P (NEXT-PRIME 0) (NEXT-PRIME (1+ P)))
     (#:G567 (RANDOM 100)))
    ((> P #:G567))
  (FORMAT T "~d " P))

T

NIL

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 

## 不同的宏是分别在何时被展开的

In [50]:
;
; 简单的情况说明
; 编译do-primes的DEFMACRO时, with-gensyms形式被展开并被编译
; 编译一个使用了do-primes的函数时, 由with-gensyms生成的代码将会运行用来生成do-primes的展开式

# HyperSpec 2.4 Standard Macro Characters

In [57]:
; 例:::
; 多层的反引用和解引用
; 生成以特定顺序仅求值特定宏参数一次的代码
(defmacro once-only ((&rest names) &body body)
    (let ((gensyms (loop for n in names collect (gensym))))
         `(let (,@(loop for g in gensyms collect `(,g (gensym))))
               `(let (,,@(loop for g in gensyms for n in names collect ``(,,g ,,n)))
                     ,(let (,@(loop for n in names for g in gensyms collect `(,n ,g)))
                           ,@body)))))

(macroexpand-1 '(once-only (start end) (list start end)))

ONCE-ONLY

(LET ((#:G577 (GENSYM)) (#:G578 (GENSYM)))
  `(LET (,`(,#:G577 ,START) ,`(,#:G578 ,END))
     ,(LET ((START #:G577) (END #:G578))
        (LIST START END))))

T


REDEFINITION-WITH-DEFMACRO: 
  redefining COMMON-LISP-USER::ONCE-ONLY in DEFMACRO


In [55]:
(defmacro do-primes8 ((var start end) &body body)
    (once-only (start end)
               `(do ((,var (next-prime ,start) (next-prime (1+ ,var))))
                    ((> ,var ,end))
                    ,@body)))

(macroexpand-1 '(do-primes8 (p 0 (random 100)) (format t "~d " p)))
(do-primes8 (p 0 (random 100)) (format t "~d " p))

DO-PRIMES8

(LET ((#:G571 0) (#:G572 (RANDOM 100)))
  (DO ((P (NEXT-PRIME #:G571) (NEXT-PRIME (1+ P))))
      ((> P #:G572))
    (FORMAT T "~d " P)))

T

NIL

2 3 5 7 11 13 17 19 23 29 31 37 41 43 